# GD12. 번역기는 대화에도 능하다
* https://www.youtube.com/watch?v=8zfYINYNS38
   
```
$ mkdir transformer_chatbot
$ sudo apt-get install g++ openjdk-8-jdk
$ sudo apt-get install curl
$ bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
$ pip install konlpy
```

## 번역 모델 만들기

In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf

import re
import os
import io
import time
import random

from sklearn.model_selection import train_test_split

# !pip install nltk # nltk가 설치되어 있지 않은 경우 주석 해제
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

import math

import gensim

import gensim.downloader as api
import random

from konlpy.tag import Mecab

In [ ]:
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip',
    origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"

with open(path_to_file, "r") as f:
    corpus = f.read().splitlines()

print("Data Size:", len(corpus))
print("Example:")

for sen in corpus[0:100][::20]: print(">>", sen)

### 토큰화
* SentencePiece: https://github.com/google/sentencepiece

In [ ]:
def generate_tokenizer(corpus,
                       vocab_size,
                       lang="spa-eng",
                       pad_id=0,
                       bos_id=1,
                       eos_id=2,
                       unk_id=3):
    file = "./%s_corpus.txt" % lang
    model = "%s_spm" % lang

    with open(file, 'w') as f:
        for row in corpus: f.write(str(row) + '\n')

    import sentencepiece as spm
    spm.SentencePieceTrainer.Train(
        '--input=./%s --model_prefix=%s --vocab_size=%d'\
        % (file, model, vocab_size) + \
        '--pad_id==%d --bos_id=%d --eos_id=%d --unk_id=%d'\
        % (pad_id, bos_id, eos_id, unk_id)
    )

    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load('%s.model' % model)

    return tokenizer

In [ ]:
cleaned_corpus = list(set(corpus))

VOCAB_SIZE = 20000
tokenizer = generate_tokenizer(cleaned_corpus, VOCAB_SIZE)
tokenizer.set_encode_extra_options("bos:eos")

In [ ]:
def preprocess_sentence(sentence): # 문장부호, 대소문자 등 정제
    sentence = sentence.lower()

    sentence = re.sub(r"([?.!,¿¡])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿¡]+", " ", sentence)

    sentence = sentence.strip()
    
    return sentence

In [ ]:
from tqdm import tqdm_notebook    # Process 과정을 보기 위해

src_corpus = []
tgt_corpus = []

for pair in tqdm_notebook(cleaned_corpus):
    src, tgt = pair.split('\t')

    src_tokens = tokenizer.encode_as_ids(preprocess_sentence(src))
    tgt_tokens = tokenizer.encode_as_ids(preprocess_sentence(tgt))
    
    # 토큰이 50개 이상인 경우 제거
    if (len(src_tokens) > 50): continue
    if (len(tgt_tokens) > 50): continue
    
    src_corpus.append(src_tokens)
    tgt_corpus.append(tgt_tokens)

len(src_corpus)

In [ ]:
enc_tensor = tf.keras.preprocessing.sequence.pad_sequences(src_corpus, padding='post')
dec_tensor = tf.keras.preprocessing.sequence.pad_sequences(tgt_corpus, padding='post')

enc_train, enc_val, dec_train, dec_val = \
train_test_split(enc_tensor, dec_tensor, test_size=0.01) # test set은 1%만

print(len(enc_train), len(enc_val), len(dec_train), len(dec_val))

### 트랜스포머 구현
* 기본 구조: https://wikidocs.net/31379
* PyTorch로 구현된 트랜스포머
    - https://paul-hyun.github.io/transformer-01/
    - https://paul-hyun.github.io/transformer-02/
    - https://paul-hyun.github.io/transformer-03/
* Attention Layer 구현: https://rubikscode.net/2019/08/05/transformer-with-python-and-tensorflow-2-0-attention-layers/
* Encoder와 Decoder 각각의 Embedding과 출력층의 Linear, 총 3개의 레이어가 Weight를 공유할 수 있게 할 것
* 하이퍼파라미터
```
transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)
```

In [68]:
# Positional Encoding 구현
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table

In [69]:
# Mask  생성하기
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(src_len, tgt_len):
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)

    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)

    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [70]:
# Multi Head Attention 구현
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        self.depth = d_model // self.num_heads

        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)

        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions


    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x


    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)

        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)

        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)

        out = self.combine_heads(out)
        out = self.linear(out)

        return out, attention_weights

In [71]:
# Position-wise Feed Forward Network 구현
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)

        return out

In [72]:
# Encoder의 레이어 구현
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):

        """
        Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, enc_attn

In [73]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, enc_out, causality_mask, padding_mask):

        """
        Masked Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        """
        Multi-Head Attention
        """
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.dec_self_attn(out, enc_out, enc_out, causality_mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

In [74]:
# Encoder 구현
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):
        out = x

        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)

        return out, enc_attns

In [75]:
# Decoder 구현
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]


    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x

        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [76]:
# Transformer 전체 모델 조립
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()

        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out


    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)

        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, causality_mask, dec_mask)

        logits = self.fc(dec_out)

        return logits, enc_attns, dec_attns, dec_enc_attns

In [78]:
# 주어진 하이퍼파라미터로 Transformer 인스턴스 생성
transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)

d_model = 512

In [79]:
# Learning Rate Scheduler 구현
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()

        self.d_model = d_model
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)

        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [80]:
# Learning Rate 인스턴스 선언 & Optimizer 구현
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

In [81]:
# Loss Function 정의
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [ ]:
# Train Step 정의
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]
    gold = tgt[:, 1:]

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

In [ ]:
# 훈련시키기
from tqdm import tqdm_notebook 

BATCH_SIZE = 64
EPOCHS = 5

for epoch in range(EPOCHS):
    total_loss = 0

    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm_notebook(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_train[idx:idx+BATCH_SIZE],
                    dec_train[idx:idx+BATCH_SIZE],
                    transformer,
                    optimizer)

        total_loss += batch_loss

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

## 번역 성능 측정하기 (1) BLEU Score
### NLTK를 활용한 BLEU Score
* BLEU Score가 50점을 넘는다는 것은 번역 품질이 높다는 뜻 (논문 제시: 20~40점)
* $(\prod_{i=1}^4 precision_i)^{\frac{1}{4}} = (\text{1-gram} \times\text{2-gram} \times\text{3-gram} \times\text{4-gram})^{\frac{1}{4}}$
    - 1-gram부터 4-gram까지의 점수 (precision) 를 모두 곱한 후, 루트를 두 번 씌우면 (^{1/4}) BLEU Score
    - 어떤 N-gram이 0의 값을 갖는다면 그 하위 N-gram 점수들이 모두 소멸되는 문제
    - 일치하는 N-gram이 없더라도 점수를 1.0 으로 유지하여 하위 점수를 보존하게끔 구현이 되어있음

In [ ]:
reference = "많 은 자연어 처리 연구자 들 이 트랜스포머 를 선호 한다".split()
candidate = "적 은 자연어 학 개발자 들 가 트랜스포머 을 선호 한다 요".split()

print("원문:", reference)
print("번역문:", candidate)
print("BLEU Score:", sentence_bleu([reference], candidate))

In [ ]:
print(sentence_bleu([reference], candidate, weights=[1, 0, 0, 0]))   # 1-gram
print(sentence_bleu([reference], candidate, weights=[0, 1, 0, 0]))   # 2-gram
print(sentence_bleu([reference], candidate, weights=[0, 0, 1, 0]))   # 3-gram
print(sentence_bleu([reference], candidate, weights=[0, 0, 0, 1]))   # 4-gram

### ```SmoothingFunction()```으로 정확한 BLEU Score 얻기

* Smoothing 함수: 모든 Precision에 아주 작은 epsilon 값을 더해주는 역할
    - 0점이 부여된 Precision도 완전한 0이 되지 않으니 점수를 1.0 으로 대체할 필요가 없어진다

In [ ]:
def calculate_bleu(reference, candidate, weights=[0.25, 0.25, 0.25, 0.25]):
    return sentence_bleu([reference],
                         candidate,
                         weights=weights,
                         smoothing_function=SmoothingFunction().method1)

print("BLEU-1:", calculate_bleu(reference, candidate, weights=[1, 0, 0, 0]))
print("BLEU-2:", calculate_bleu(reference, candidate, weights=[0, 1, 0, 0]))
print("BLEU-3:", calculate_bleu(reference, candidate, weights=[0, 0, 1, 0]))
print("BLEU-4:", calculate_bleu(reference, candidate, weights=[0, 0, 0, 1]))

print("\nBLEU-Total:", calculate_bleu(reference, candidate))

### 트랜스포머 모델의 번역 성능 알아보기

In [ ]:
# translate()

def evaluate(sentence, model, src_tokenizer, tgt_tokenizer):
    sentence = preprocess_sentence(sentence)

    pieces = src_tokenizer.encode_as_pieces(sentence)
    tokens = src_tokenizer.encode_as_ids(sentence)

    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')
    
    ids = []
    output = tf.expand_dims([tgt_tokenizer.bos_id()], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if tgt_tokenizer.eos_id() == predicted_id:
            result = tgt_tokenizer.decode_ids(ids)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = tgt_tokenizer.decode_ids(ids)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

def translate(sentence, model, src_tokenizer, tgt_tokenizer):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = \
    evaluate(sentence, model, src_tokenizer, tgt_tokenizer)

    return result

In [ ]:
def eval_bleu(src_corpus, tgt_corpus, verbose=True):
    total_score = 0.0
    sample_size = len(tgt_corpus)

    for idx in tqdm_notebook(range(sample_size)):
        src_tokens = src_corpus[idx]
        tgt_tokens = tgt_corpus[idx]

        src_sentence = tokenizer.decode_ids((src_tokens.tolist()))
        tgt_sentence = tokenizer.decode_ids((tgt_tokens.tolist()))

        reference = preprocess_sentence(tgt_sentence).split()
        candidate = translate(src_sentence, transformer, tokenizer, tokenizer).split()

        score = sentence_bleu([reference], candidate,
                              smoothing_function=SmoothingFunction().method1)
        total_score += score

        if verbose:
            print("Source Sentence: ", src_sentence)
            print("Model Prediction: ", candidate)
            print("Real: ", reference)
            print("Score: %lf\n" % score)

    print("Num of Sample:", sample_size)
    print("Total Score:", total_score / sample_size)

In [ ]:
eval_bleu(enc_val[:3], dec_val[:3], True)

In [ ]:
# 전체 테스트셋으로 측정하는 것은 시간이 제법 걸리니 1/10만 사용해서 실습하는 걸 권장
# enc_val[::10] 의 [::10] 은 리스트를 10개씩 건너뛰어 추출하라는 의미
eval_bleu(enc_val[::10], dec_val[::10], verbose=False)

## 번역 성능 측정하기 (2) Beam Search Decoder

In [ ]:
def beam_search_decoder(prob, beam_size):
    sequences = [[[], 1.0]]  # 생성된 문장과 점수를 저장

    for tok in prob:
        all_candidates = []

        for seq, score in sequences:
            for idx, p in enumerate(tok): # 각 단어의 확률을 총점에 누적 곱
                candidate = [seq + [idx], score * -math.log(-(p-1))]
                all_candidates.append(candidate)

        ordered = sorted(all_candidates,
                         key=lambda tup:tup[1],
                         reverse=True) # 총점 순 정렬
        sequences = ordered[:beam_size] # Beam Size에 해당하는 문장만 저장 

    return sequences

In [ ]:
vocab = {
    0: "<pad>",
    1: "까요?",
    2: "커피",
    3: "마셔",
    4: "가져",
    5: "될",
    6: "를",
    7: "한",
    8: "잔",
    9: "도",
}

prob_seq = [[0.01, 0.01, 0.60, 0.32, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.75, 0.01, 0.01, 0.17],
            [0.01, 0.01, 0.01, 0.35, 0.48, 0.10, 0.01, 0.01, 0.01, 0.01],
            [0.24, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.68],
            [0.01, 0.01, 0.12, 0.01, 0.01, 0.80, 0.01, 0.01, 0.01, 0.01],
            [0.01, 0.81, 0.01, 0.01, 0.01, 0.01, 0.11, 0.01, 0.01, 0.01],
            [0.70, 0.22, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],
            [0.91, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]]

prob_seq = np.array(prob_seq)
beam_size = 3

result = beam_search_decoder(prob_seq, beam_size)

for seq, score in result:
    sentence = ""

    for word in seq:
        sentence += vocab[word] + " "

    print(sentence, "// Score: %.4f" % score)

### Beam Search Decoder 작성 및 평가하기

In [ ]:
# beam_search_decoder() 구현
def calc_prob(src_ids, tgt_ids, model):
    enc_padding_mask, combined_mask, dec_padding_mask = \
    generate_masks(src_ids, tgt_ids)

    predictions, enc_attns, dec_attns, dec_enc_attns =\
    model(src_ids, 
            tgt_ids,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask)

    return tf.math.softmax(predictions, axis=-1)
  
def beam_search_decoder(sentence, 
                        src_len,
                        tgt_len,
                        model,
                        src_tokenizer,
                        tgt_tokenizer,
                        beam_size):
    sentence = preprocess_sentence(sentence)

    pieces = src_tokenizer.encode_as_pieces(sentence)
    tokens = src_tokenizer.encode_as_ids(sentence)

    src_in = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                            maxlen=src_len,
                                                            padding='post')

    pred_cache = np.zeros((beam_size * beam_size, tgt_len), dtype=np.long)
    pred = np.zeros((beam_size, tgt_len), dtype=np.long)

    eos_flag = np.zeros((beam_size, ), dtype=np.long)
    scores = np.ones((beam_size, ))

    pred[:, 0] = tgt_tokenizer.bos_id()

    dec_in = tf.expand_dims(pred[0, :1], 0)
    prob = calc_prob(src_in, dec_in, model)[0, -1].numpy()

    for seq_pos in range(1, tgt_len):
        score_cache = np.ones((beam_size * beam_size, ))

        # init
        for branch_idx in range(beam_size):
            cache_pos = branch_idx*beam_size

            score_cache[cache_pos:cache_pos+beam_size] = scores[branch_idx]
            pred_cache[cache_pos:cache_pos+beam_size, :seq_pos] = \
            pred[branch_idx, :seq_pos]

        for branch_idx in range(beam_size):
            cache_pos = branch_idx*beam_size

            if seq_pos != 1:   # 모든 Branch를 <BOS>로 시작하는 경우를 방지
                dec_in = pred_cache[branch_idx, :seq_pos]
                dec_in = tf.expand_dims(dec_in, 0)

                prob = calc_prob(src_in, dec_in, model)[0, -1].numpy()

            for beam_idx in range(beam_size):
                max_idx = np.argmax(prob)

                score_cache[cache_pos+beam_idx] *= prob[max_idx]
                pred_cache[cache_pos+beam_idx, seq_pos] = max_idx

                prob[max_idx] = -1

        for beam_idx in range(beam_size):
            if eos_flag[beam_idx] == -1: continue

            max_idx = np.argmax(score_cache)
            prediction = pred_cache[max_idx, :seq_pos+1]

            pred[beam_idx, :seq_pos+1] = prediction
            scores[beam_idx] = score_cache[max_idx]
            score_cache[max_idx] = -1

            if prediction[-1] == tgt_tokenizer.eos_id():
                eos_flag[beam_idx] = -1

    return pred

In [ ]:
# beam_bleu() 구현
def calculate_bleu(reference, candidate, weights=[0.25, 0.25, 0.25, 0.25]):
    return sentence_bleu([reference],
                            candidate,
                            weights=weights,
                            smoothing_function=SmoothingFunction().method1)

def beam_bleu(reference, ids, tokenizer):
    reference = reference.split()

    total_score = 0.0
    for _id in ids:
        candidate = tokenizer.decode_ids(_id.tolist()).split()
        score = calculate_bleu(reference, candidate)

        print("Reference:", reference)
        print("Candidate:", candidate)
        print("BLEU:", calculate_bleu(reference, candidate))

        total_score += score

    return total_score / len(ids)

In [ ]:
idx = 324

ids = \
beam_search_decoder(tokenizer.decode_ids(enc_val[idx].tolist()),
                    enc_train.shape[-1],
                    dec_train.shape[-1],
                    transformer,
                    tokenizer,
                    tokenizer,
                    beam_size=5)

bleu = beam_bleu(tokenizer.decode_ids(dec_val[idx].tolist()), ids, tokenizer)

## 데이터 부풀리기
```
$ pip install gensim
```
   
* gensim에 사전 훈련된 embedding 모델을 불러오는 두 가지 방법
    - 직접 모델을 다운로드해 load 하는 방법
    - gensim 이 자체적으로 지원하는 downloader 를 활용해 모델을 load 하는 방법 (한국어는 gensim이 지원하지 않기 때문에 불가)
* https://github.com/RaRe-Technologies/gensim-data
* 대표적으로 사용되는 embedding 모댈: word2vec-google-news-300 (시간 너무 많이 걸림)
* 아래에서 사용하는 모델: (적당한 사이즈의) glove-wiki-gigaword-300

In [ ]:
wv = api.load('glove-wiki-gigaword-300')

In [ ]:
wv.most_similar("banana")

'''
[('bananas', 0.6691170930862427),
 ('mango', 0.580410361289978),
 ('pineapple', 0.5492372512817383),
 ('coconut', 0.5462779402732849),
 ('papaya', 0.541056752204895),
 ('fruit', 0.52181077003479),
 ('growers', 0.4877638518810272),
 ('nut', 0.48399588465690613),
 ('peanut', 0.4806201756000519),
 ('potato', 0.4806118309497833)]
'''

In [ ]:
sample_sentence = "you know ? all you need is attention ."
sample_tokens = sample_sentence.split()

selected_tok = random.choice(sample_tokens)

result = ""
for tok in sample_tokens:
    if tok is selected_tok:
        result += wv.most_similar(tok)[0][0] + " "

    else:
        result += tok + " "

print("From:", sample_sentence)
print("To:", result)

'''
From: you know ? all you need is attention .
To: you know ? all you need is attention , 
'''

### Lexical Substitution 구현하기

In [ ]:
# Lexical Substitution 구현하기
def lexical_sub(sentence, word2vec):
    import random

    res = ""
    toks = sentence.split()

    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[0][0]

    except:   # 단어장에 없는 단어
        return None

    for tok in toks:
        if tok is _from: res += _to + " "
        else: res += tok + " "

    return res

from tqdm import tqdm_notebook

new_corpus = []

for idx in tqdm_notebook(range(3000)):
    old_src = tokenizer.decode_ids(src_corpus[idx])

    new_src = lexical_sub(old_src, wv)

    if new_src is not None: new_corpus.append(new_src)

    new_corpus.append(old_src)

print(new_corpus[:10])

## 프로젝트: 멋진 챗봇 만들기

### 데이터 다운로드
* https://github.com/songys/Chatbot_data

In [52]:
data = pd.read_csv('ChatbotData.csv')
data.head(10)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


### 데이터 정졔

In [53]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 영문자를 모두 소문자로 변환
    sentence = re.sub('[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣?.!,1-9\\s]', "", sentence) # 영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 정규식을 활용하여 모두 제거
    sentence = sentence.strip()
    
    return sentence

In [54]:
print(preprocess_sentence("3박4일 정도 놀러가고 싶다~~~~~~~~$*$&@($_)hello"))

3박4일 정도 놀러가고 싶다hello


### 데이터 토큰화

In [55]:
questions = data['Q'] # source
answers = data['A'] # target

def build_corpus(src, tgt):
    que_corpus = []
    ans_corpus = []
    
    mecab = Mecab()
    
    for idx, sentence in enumerate(src):
        src_preprocessed = preprocess_sentence(sentence)
        tgt_preprocessed = preprocess_sentence(tgt[idx])
        
        if src_preprocessed in que_corpus or tgt_preprocessed in ans_corpus:
            continue
        else:
            src_tokenized = mecab.morphs(src_preprocessed)
            tgt_tokenized = mecab.morphs(tgt_preprocessed)
            
            # 토큰 길이는 각 최대 20으로 설정
            if len(src_tokenized) <= 20 and len(tgt_tokenized) <= 20:
                que_corpus.append(src_tokenized)
                ans_corpus.append(tgt_tokenized)
            else:
                continue
    
    return que_corpus, ans_corpus

que_corpus, ans_corpus = build_corpus(questions, answers)

# 둘이 같아야 함
print(len(que_corpus))
print(len(ans_corpus))

11672
11672


In [56]:
que_corpus[:10]

[['12', '시', '땡', '!'],
 ['1', '지망', '학교', '떨어졌', '어'],
 ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'],
 ['3', '박', '4', '일', '정도', '놀', '러', '가', '고', '싶', '다'],
 ['ppl', '심하', '네'],
 ['sd', '카드', '망가졌', '어'],
 ['sd', '카드', '안', '돼'],
 ['sns', '맞', '팔', '왜', '안', '하', '지', 'ㅠㅠ'],
 ['sns', '시간', '낭비', '인', '거', '아', '는데', '매일', '하', '는', '중'],
 ['sns', '시간', '낭비', '인데', '자꾸', '보', '게', '됨']]

In [57]:
ans_corpus[:10]

[['하루', '가', '또', '가', '네요', '.'],
 ['위로', '해', '드립니다', '.'],
 ['여행', '은', '언제나', '좋', '죠', '.'],
 ['여행', '은', '언제나', '좋', '죠', '.'],
 ['눈살', '이', '찌푸려', '지', '죠', '.'],
 ['다시', '새로', '사', '는', '게', '마음', '편해요', '.'],
 ['다시', '새로', '사', '는', '게', '마음', '편해요', '.'],
 ['잘', '모르', '고', '있', '을', '수', '도', '있', '어요', '.'],
 ['시간', '을', '정하', '고', '해', '보', '세요', '.'],
 ['시간', '을', '정하', '고', '해', '보', '세요', '.']]

### Augmentation
* https://github.com/Kyubyong/wordvectors

In [58]:
word2vec = gensim.models.Word2Vec.load('ko.bin')

def lexical_sub(sentence, word2vec):
    import random

    res = ""
    # toks = sentence.split()
    toks = sentence

    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[0][0]

    except:   # 단어장에 없는 단어
        return None

    for tok in toks:
        if tok is _from: res += _to + " "
        else: res += tok + " "

    return res

# 테스트
mecab = Mecab()
print("원본:", que_corpus[0])
print("결과:", lexical_sub(que_corpus[0], word2vec))

원본: ['12', '시', '땡', '!']
결과: 12 시 끗 ! 


/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if sys.path[0] == '':


In [59]:
from tqdm import tqdm_notebook

new_que_corpus = []
new_ans_corpus = []

# Augmentation된 que_corpus 와 원본 ans_corpus 가 병렬을 이루도록
for idx in tqdm_notebook(range(len(que_corpus))):
    que_augmented = lexical_sub(que_corpus[idx], word2vec)
    ans = ans_corpus[idx]
    
    if que_augmented is not None:
        new_que_corpus.append(que_augmented.split())
        new_ans_corpus.append(ans)
        # print("[que]", que_corpus[idx], "->", que_augmented, "/", ans)
    else:
        # print("[que] Augmentation is None:", que_corpus[idx], "/", ans_corpus[idx])
        continue
    
for idx in tqdm_notebook(range(len(ans_corpus))):
    que = que_corpus[idx]
    ans_augmented = lexical_sub(ans_corpus[idx], word2vec)
    
    if ans_augmented is not None:
        new_que_corpus.append(que)
        new_ans_corpus.append(ans_augmented.split())
        # print("[ans]", que, "/", ans_corpus[idx], "->", ans_augmented)
    else:
        # print("[ans] Augmentation is None:", que_corpus[idx], "/", ans_corpus[idx])
        continue
    

print(len(new_que_corpus))
print(len(new_ans_corpus), '\n')

print(new_que_corpus[0])
print(new_ans_corpus[0], '\n')

print(new_que_corpus[56])
print(new_ans_corpus[56], '\n')

que_corpus = que_corpus + new_que_corpus
ans_corpus = ans_corpus + new_ans_corpus

print(len(que_corpus))
print(len(ans_corpus), '\n')

/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if sys.path[0] == '':


/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



20176
20176 

['12', '시', '끗', '!']
['하루', '가', '또', '가', '네요', '.'] 

['강아지', '키울', '가능성', '있', '을까']
['먼저', '생활', '패턴', '을', '살펴', '보', '세요', '.'] 

31848
31848 



### 데이터 벡터화
* 타겟 데이터인 ```ans_corpus``` 에 ```<start>```, ```<end>``` 토큰 추가

In [62]:
tgt_corpus = []

for corpus in ans_corpus:
    tgt_corpus.append(["<start>"] + corpus + ["<end>"])
    
print(tgt_corpus[0])
print(tgt_corpus[325])

ans_corpus = tgt_corpus

['<start>', '하루', '가', '또', '가', '네요', '.', '<end>']
['<start>', '이야기', '를', '해', '보', '세요', '.', '<end>']


In [63]:
# 단어 사전 생성
from collections import Counter

voc_data = que_corpus + ans_corpus

words = np.concatenate(voc_data).tolist()
counter = Counter(words)
counter = counter.most_common(30000-2)
vocab = ['<pad>', '<unk>'] + [key for key, _ in counter]
word_to_index = {word:index for index, word in enumerate(vocab)}
index_to_word = {index:word for word, index in word_to_index.items()}

word_to_index

{'<pad>': 0,
 '<unk>': 1,
 '.': 2,
 '<start>': 3,
 '<end>': 4,
 '이': 5,
 '하': 6,
 '는': 7,
 '을': 8,
 '세요': 9,
 '가': 10,
 '어': 11,
 '고': 12,
 '좋': 13,
 '해': 14,
 '거': 15,
 '있': 16,
 '보': 17,
 '은': 18,
 '지': 19,
 '?': 20,
 '도': 21,
 '아': 22,
 '나': 23,
 '게': 24,
 '겠': 25,
 '는데': 26,
 '에': 27,
 '예요': 28,
 '사람': 29,
 '를': 30,
 '어요': 31,
 '다': 32,
 '같': 33,
 '죠': 34,
 '한': 35,
 '싶': 36,
 '없': 37,
 '사랑': 38,
 '네요': 39,
 '면': 40,
 '것': 41,
 '수': 42,
 '안': 43,
 '네': 44,
 '친구': 45,
 '봐요': 46,
 '의': 47,
 '잘': 48,
 '아요': 49,
 '생각': 50,
 '말': 51,
 '않': 52,
 '할': 53,
 '마음': 54,
 '되': 55,
 '너무': 56,
 '주': 57,
 '했': 58,
 '만': 59,
 '더': 60,
 '일': 61,
 '기': 62,
 '내': 63,
 '었': 64,
 '들': 65,
 '연락': 66,
 '이별': 67,
 '남자': 68,
 '여자': 69,
 '힘들': 70,
 '많이': 71,
 '해요': 72,
 '시간': 73,
 '으면': 74,
 '남': 75,
 '한테': 76,
 '길': 77,
 '에요': 78,
 '먹': 79,
 '썸': 80,
 '괜찮': 81,
 '때': 82,
 '좀': 83,
 '으로': 84,
 '에서': 85,
 '았': 86,
 '많': 87,
 '짝': 88,
 '야': 89,
 '저': 90,
 '!': 91,
 '요': 92,
 '받': 93,
 '건': 94,
 '뭐': 95,
 '만나'

In [64]:
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index[word] if word in word_to_index else word_to_index['<unk>'] for word in sentence]

def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<unk>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

def vectorize(corpus, word_to_index):
    data = []
    for sen in corpus:
        sen = get_encoded_sentence(sen, word_to_index)
        data.append(sen)
    return data

que_train = vectorize(que_corpus, word_to_index)
ans_train = vectorize(ans_corpus, word_to_index)

print(len(que_train))
print(len(ans_train))
print(que_train[0])
print(ans_train[0])

31848
31848
[2539, 182, 4106, 91]
[3, 271, 10, 157, 10, 39, 2, 4]


In [65]:
enc_tensor = tf.keras.preprocessing.sequence.pad_sequences(que_train, padding='post')
dec_tensor = tf.keras.preprocessing.sequence.pad_sequences(ans_train, padding='post')

enc_train, enc_val, dec_train, dec_val = \
train_test_split(enc_tensor, dec_tensor, test_size=0.01) # test set은 1%만

print(len(enc_train), len(enc_val), len(dec_train), len(dec_val))

31529 319 31529 319


In [66]:
enc_train[0]

array([ 231,  218,    7,   41,   33,  113,   63,   10, 1746,  190,   27,
        187,  296,   37,    7, 1781,   24,  144,    2,    0], dtype=int32)

In [67]:
dec_train[0]

array([   3,   38,  342,   27, 1746,   18, 3617,    5,   37,   31,    2,
          4,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

### train
* 위 실습의 Transformer 그대로 사용

In [82]:
# 주어진 하이퍼파라미터로 Transformer 인스턴스 생성
VOCAB_SIZE = 20000

transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)

d_model = 512

In [83]:
# Learning Rate 인스턴스 선언 & Optimizer 구현
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

In [84]:
# Train Step 정의
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]
    gold = tgt[:, 1:]

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

In [85]:
# 훈련시키기
from tqdm import tqdm_notebook 

BATCH_SIZE = 64
EPOCHS = 30

for epoch in range(EPOCHS):
    total_loss = 0

    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm_notebook(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_train[idx:idx+BATCH_SIZE],
                    dec_train[idx:idx+BATCH_SIZE],
                    transformer,
                    optimizer)

        total_loss += batch_loss

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

/home/aiffel0042/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


### test

In [88]:
# translate()

def evaluate(sentence, model):
    mecab = Mecab()
    
    sentence = preprocess_sentence(sentence)
    pieces = mecab.morphs(sentence)
    
    tokens = []
    for sen in pieces:
        sen= get_encoded_sentence(sen, word_to_index)
        tokens.append(sen)
    
    _input = tf.keras.preprocessing.sequence.pad_sequences(tokens,
                                                        value=word_to_index["<pad>"],
                                                        padding='pre',
                                                        maxlen=20)
    
    ids = []
    output = tf.expand_dims([word_to_index["<start>"]], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if word_to_index["<end>"] == predicted_id:
            result = get_decoded_sentence(ids, index_to_word)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = get_decoded_sentence(ids, index_to_word)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

def translate(sentence, model):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = \
    evaluate(sentence, model)

    return result

In [90]:
sample_sentences = [
    "지루하다, 놀러가고 싶어.",
    "오늘 일찍 일어났더니 피곤하다.",
    "간만에 여자친구랑 데이트 하기로 했어.",
    "집에 있는다는 소리야.",
]

for sentence in sample_sentences:
    print("source:", sentence)
    print("answered:", translate(sentence, transformer))

source: 지루하다, 놀러가고 싶어.
answered: 정말 참 아 했 던 만큼 더 나 봐요 는데
source: 오늘 일찍 일어났더니 피곤하다.
answered: 만 더 버텨 보 세요 .
source: 간만에 여자친구랑 데이트 하기로 했어.
answered: 하 거나 직접 만나 보 세요 .
source: 집에 있는다는 소리야.
answered: 하 고 출퇴근 하 고 싶 네요 .
